In [14]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from warnings import filterwarnings
filterwarnings('ignore')

In [15]:
df= pd.read_csv('sales-data.csv',header=None,names=['Month','Sales'])
df.head()

,Month,Sales
0,1980-01,154
1,1980-02,96
2,1980-03,73
3,1980-04,49
4,1980-05,36


In [17]:
df['Month'] = pd.to_datetime(df['Month'],format='%Y-%m')
df.set_index('Month',inplace=True)

In [18]:
df.head()

,Sales
Month,
1980-01-01,154
1980-02-01,96
1980-03-01,73
1980-04-01,49
1980-05-01,36


In [19]:
train_len=60
train = df[:train_len]
test = df[train_len:]

### Box cox transformation

In [20]:
from scipy.stats import boxcox
df_boxcox= pd.Series(boxcox(df['Sales'],lmbda=0),index=df.index)

In [73]:
train_df_boxcox = df_boxcox[:train_len]
test_df_boxcox = df_boxcox[train_len:]

### First differencing

In [21]:
df_boxcox_diff = pd.Series(df_boxcox - df_boxcox.shift(),index = df.index)
df_boxcox_diff.dropna(inplace=True)

In [22]:
df_boxcox_diff

Month
1980-02-01   -0.472604
1980-03-01   -0.273889
1980-04-01   -0.398639
1980-05-01   -0.308301
1980-06-01    0.494019
                ...   
1986-01-01   -0.055742
1986-02-01   -0.712440
1986-03-01    0.050644
1986-04-01   -0.267315
1986-05-01    0.092373
Length: 76, dtype: float64

In [23]:
train_boxcox_diff = df_boxcox_diff[:train_len-1]
test_boxcox_diff = df_boxcox_diff[train_len-1:]

### Auto Regression(AR) model 

In [24]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(train_boxcox_diff,order = (8,0,0))
model_fit = model.fit()
print(model_fit.params)

const     0.021115
ar.L1    -0.430993
ar.L2     0.027240
ar.L3    -0.104778
ar.L4    -0.113528
ar.L5    -0.375331
ar.L6    -0.327879
ar.L7    -0.496533
ar.L8    -0.361045
sigma2    0.032236
dtype: float64


### Recover original time series forecast

In [35]:
y_hat_ar = df_boxcox_diff.copy()
y_hat_ar['ar_boxcox_diff'] = model_fit.predict(df_boxcox_diff.index.min(),df_boxcox_diff.index.max())
y_hat_ar['ar_boxcox'] = y_hat_ar['ar_boxcox_diff'].cumsum()
y_hat_ar['ar_boxcox'] = y_hat_ar['ar_boxcox'].add(df_boxcox[0])
## Inverse boxcox
from scipy.special import inv_boxcox
y_hat_ar['ar_boxcox'] = inv_boxcox(y_hat_ar['ar_boxcox'],0)
y_hat_ar['ar_boxcox']

1980-02-01    157.286350
1980-03-01    134.180635
1980-04-01    107.949169
1980-05-01    101.292148
1980-06-01     97.509718
                 ...    
1986-01-01    907.418177
1986-02-01    688.813636
1986-03-01    528.696270
1986-04-01    443.868667
1986-05-01    419.115627
Freq: MS, Name: predicted_mean, Length: 76, dtype: float64

In [36]:
df

,Sales
Month,
1980-01-01,154
1980-02-01,96
1980-03-01,73
1980-04-01,49
1980-05-01,36
...,...
1986-01-01,628
1986-02-01,308
1986-03-01,324


In [39]:
## Checking out the rmse and mape
from sklearn.metrics import mean_squared_error
rmse  = round(np.sqrt(mean_squared_error(test['Sales'],y_hat_ar['ar_boxcox'][test.index.min():])),2)
mape = round(np.mean((np.abs(test['Sales']-y_hat_ar['ar_boxcox'][test.index.min():]))/test['Sales']*100),2)
print(rmse,mape)

212.74 47.05


### Moving Average(MA) model

In [40]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(train_boxcox_diff,order = (0,0,3))
model_fit = model.fit()
print(model_fit.params)

const     0.025926
ma.L1    -0.125552
ma.L2    -0.285459
ma.L3    -0.588185
sigma2    0.094206
dtype: float64


### Recover original time series forecast data

In [47]:
y_hat_ma = df_boxcox_diff.copy()
y_hat_ma['ma_boxcox_diff'] = model_fit.predict(df_boxcox_diff.index.min(),df_boxcox_diff.index.max())
y_hat_ma['ma_boxcox_diff'] = y_hat_ma['ma_boxcox_diff'].cumsum()
y_hat_ma['ma_boxcox'] = y_hat_ma['ma_boxcox_diff'].add(df_boxcox[0])
##inv boxcox
y_hat_ma['ma_forecast'] = inv_boxcox(y_hat_ma['ma_boxcox'],0)

In [48]:
## Checking out the rmse and mape
from sklearn.metrics import mean_squared_error
rmse  = round(np.sqrt(mean_squared_error(test['Sales'],y_hat_ma['ma_forecast'][test.index.min():])),2)
mape = round(np.mean((np.abs(test['Sales']-y_hat_ma['ma_forecast'][test.index.min():]))/test['Sales']*100),2)
print(rmse,mape)

830.6 238.99


### ARMA Method

In [49]:
model= ARIMA(df_boxcox_diff,order =(4,0,3))
model_fit = model.fit()
print(model_fit.params)

const     0.019250
ar.L1     0.256620
ar.L2     1.048790
ar.L3    -0.605033
ar.L4    -0.491620
ma.L1    -0.760580
ma.L2    -0.668140
ma.L3     0.757117
sigma2    0.033200
dtype: float64


In [51]:
#Recovering the original time series format
y_hat_arma = df_boxcox_diff.copy()
y_hat_arma['arma_boxcox_diff'] = model_fit.predict(df_boxcox_diff.index.min(),df_boxcox_diff.index.max())
y_hat_arma['arma_boxcox'] = y_hat_arma['arma_boxcox_diff'].cumsum()
y_hat_arma['arma_boxcox'] = y_hat_arma['arma_boxcox'].add(df_boxcox[0])
y_hat_arma['arma_forecast'] = inv_boxcox(y_hat_arma['arma_boxcox'],0.2)

In [58]:
## RMSE and MAPE
rmse =round(np.sqrt( mean_squared_error(test['Sales'],y_hat_arma['arma_forecast'][test.index.min():])),2)
mape = round(np.mean((np.abs(test['Sales']-y_hat_arma['arma_forecast'][test.index.min():]))/test['Sales']*100),2)
print(rmse,mape)

445.71 85.6


### ARIMA model

In [74]:
model = ARIMA(train_df_boxcox,order = (4,1,3))
model_fit = model.fit()
print(model_fit.params)

ar.L1     0.213314
ar.L2     1.126938
ar.L3    -0.611181
ar.L4    -0.524832
ma.L1    -0.783944
ma.L2    -0.792938
ma.L3     0.988796
sigma2    0.030451
dtype: float64


In [86]:
## Recovering the original time series
y_hat_arima = df_boxcox_diff.copy()
y_hat_arima['arima_boxcox_diff'] = model_fit.predict(start=df_boxcox_diff.index.min(),end=df_boxcox_diff.index.max())
y_hat_arima['arima_boxcox'] = y_hat_arima['arima_boxcox_diff'].cumsum()
y_hat_arima['arima_boxcox'] = y_hat_arima['arima_boxcox'].add(df_boxcox[0])
y_hat_arima['arima_forecast'] = inv_boxcox(y_hat_arima['arima_boxcox'],1.1)

In [87]:
## Checking out the rmse and mape
from sklearn.metrics import mean_squared_error
rmse  = round(np.sqrt(mean_squared_error(test['Sales'],y_hat_arima['arima_forecast'][test.index.min():])),2)
mape = round(np.mean((np.abs(test['Sales']-y_hat_arima['arima_forecast'][test.index.min():]))/test['Sales']*100),2)
print(rmse,mape)

304.71 38.01


### SARIMA 

In [88]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(df_boxcox,order=(7,2,9),seasonal_order=(3,1,1,12))
model_fit = model.fit()
print(model_fit.params)

ar.L1      -1.392610
ar.L2      -0.589887
ar.L3      -0.173929
ar.L4      -0.256081
ar.L5      -0.522608
ar.L6      -0.641861
ar.L7      -0.304727
ma.L1      -0.078722
ma.L2      -0.825075
ma.L3      -0.060318
ma.L4       0.115576
ma.L5       0.168881
ma.L6      -0.071636
ma.L7      -0.875165
ma.L8      -0.192480
ma.L9       0.862965
ar.S.L12   -0.740469
ar.S.L24   -0.629054
ar.S.L36   -0.605924
ma.S.L12   -0.028449
sigma2      0.016991
dtype: float64


In [90]:
## Recovering the original time series
y_hat_sarima = df_boxcox.copy()
y_hat_sarima['sarima_boxcox'] = model_fit.predict(df_boxcox_diff.index.min(),df_boxcox_diff.index.max())
y_hat_sarima['sarima_forecast'] = inv_boxcox(y_hat_sarima['sarima_boxcox'],0)

In [91]:
## Checking out the rmse and mape
from sklearn.metrics import mean_squared_error
rmse  = round(np.sqrt(mean_squared_error(test['Sales'],y_hat_sarima['sarima_forecast'][test.index.min():])),2)
mape = round(np.mean((np.abs(test['Sales']-y_hat_sarima['sarima_forecast'][test.index.min():]))/test['Sales']*100),2)
print(rmse,mape)

41.98 9.49
